In [1]:
import os
import shutil

# Скачиваем все .csv файлы
output_dir = '.'
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        full_path = os.path.join(dirname, filename)
        if filename != 'sample_submission.csv':
            shutil.copy(full_path, output_dir)
            print(f'Скачан файл: {filename}')

Скачан файл: personality_datasert.csv
Скачан файл: personality_dataset.csv
Скачан файл: train.csv
Скачан файл: test.csv


In [2]:
# Скачиваем дополнительные модули из репо
!wget -q https://raw.githubusercontent.com/saspav/python_for_pro/main/final_cat/some_functions_clf.py > /dev/null 2>&1

In [3]:
!pip install py-boost -q 2>/dev/null

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 76.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00


In [4]:
import re
import random
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor


from py_boost import GradientBoosting

import warnings

warnings.filterwarnings("ignore")

from some_functions_clf import (SEED, make_train_valid, DataTransform, 
                                train_valid_model, make_submit)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
# Зафиксируем сиды
np.random.seed(SEED)
random.seed(SEED)

# Целевая переменная
target = 'Personality'

train, valid, test = make_train_valid(add_original_df=True)

imputer, imputer_params = IterativeImputer, dict(
    initial_strategy='most_frequent',
    min_value=0,
    max_value=15,
    skip_complete=True,
    max_iter=10,
    random_state=SEED,
)

dts = DataTransform(set_category=False, set_num_int=False,
                    preprocessor=imputer, **imputer_params)

fill_nan_cat = True

# Применяем трансформации
train = dts.fit_transform(train, fill_nan_cat=fill_nan_cat)
valid = dts.transform(valid, fill_nan_cat=fill_nan_cat)

# Трансформируем признаки тестовой выборки
test = dts.transform(test, fill_nan_cat=fill_nan_cat)

# Колонки для моделей
model_columns = dts.all_features

In [6]:
pb_best_grid = dict(loss='bce', metric='auc',
                    ntrees=1000, max_depth=4, verbose=100,
                    lr=.01, lambda_l2=1, subsample=.8, colsample=.8,
                    min_data_in_leaf=10, min_gain_to_split=0, max_bin=256)

pb, metrics, opt_thrs = train_valid_model(GradientBoosting, 'PB', pb_best_grid,
                                          train, valid, model_columns, target)
metrics

[15:35:32] Stdout logging level is INFO.
[15:35:32] GDBT train starts. Max iter 1000, early stopping rounds 100
[15:35:41] Iter 0; Sample 0, AUC = 0.9612745357588593; 
[15:35:43] Iter 100; Sample 0, AUC = 0.9682791876252789; 
[15:35:44] Iter 200; Sample 0, AUC = 0.9688994364812223; 
[15:35:45] Iter 300; Sample 0, AUC = 0.9696628342347112; 
[15:35:46] Iter 400; Sample 0, AUC = 0.9702740062781287; 
[15:35:47] Iter 500; Sample 0, AUC = 0.9706743315305775; 
[15:35:48] Iter 600; Sample 0, AUC = 0.9707401384213911; 
[15:35:48] Early stopping at iter 616, best iter 516, best_score 0.9707881698876745
Оптимальный порог: 0.5284


,Metric,TrainPB,ValidPB,"DiffPB,%"
0,accuracy,0.971456,0.969501,-0.20
1,precision,0.952118,0.952229,0.01
2,recall,0.937565,0.929534,-0.86
3,f1,0.944785,0.940745,-0.43
4,roc_auc,0.980322,0.970788,-0.97


In [7]:
# threshold = opt_thrs
threshold = 0.5

params_imputer = sorted(imputer_params.items())[:2]
submit_postfix = f"_{str(imputer).split('.')[-1][:-2]}-{params_imputer}-thr={threshold}"
submit_postfix = '_'.join(re.findall(r'\w+', submit_postfix))
submit_postfix = submit_postfix + ('', '_nan_cat')[fill_nan_cat]

# Вызываем функцию для формирования сабмита: передаем обученную модель
file_submit = make_submit(pb, test[model_columns], threshold, 
                          dts.reverse_mapping, postfix=submit_postfix)

shutil.copy(file_submit, 'submission.csv')

Сформирован файл сабмита: submit_GradientBoosting_IterativeImputer_initial_strategy_most_frequent_max_iter_10_thr_0_5_nan_cat.csv


'submission.csv'